<a href="https://colab.research.google.com/github/vijayshankarrealdeal/intro_to_pytorch-Gans/blob/main/Boltzmachine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip

--2021-08-03 19:54:24--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.2’

ml-100k.zip.2       100%[===================>]   4.70M  26.5MB/s    in 0.2s    

2021-08-03 19:54:25 (26.5 MB/s) - ‘ml-100k.zip.2’ saved [4924029/4924029]

--2021-08-03 19:54:25--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.2’

ml-1m.zip.2         100%[===================>]   5.64M  30.6MB/s    in 0.2s    

2021-08-03 19:54:25 (30.6 MB/s) - ‘ml-1m.zip.2’

In [119]:
import pandas as pd 
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data 

In [120]:
import zipfile 

In [121]:
def extract(filepath,filedes):
  with zipfile.ZipFile(filepath) as zip_ref:
    zip_ref.extractall(filedes)

extract('/content/ml-100k.zip','/content/data')
extract('/content/ml-1m.zip','/content/data')

In [122]:
movies = pd.read_csv('/content/data/ml-1m/movies.dat',sep = '::',header=None,engine = 'python',encoding = 'latin-1')

In [123]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [124]:
users = pd.read_csv('/content/data/ml-1m/users.dat',sep = '::',header=None,engine = 'python',encoding = 'latin-1')

In [125]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [126]:
rating = pd.read_csv('/content/data/ml-1m/ratings.dat',sep = '::',header=None,engine = 'python',encoding = 'latin-1')

In [127]:
rating.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [128]:

training_set = pd.read_csv('/content/data/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('/content/data/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')


nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))


def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)


training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [129]:
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [130]:
nv,nh

(1682, 100)

In [131]:
np_epoch = 10
for epoch in range(1,np_epoch+1):
  train_loss = 0
  s = 0.0
  for id_user in range(0,nb_users - batch_size,batch_size):
    vk = train_set[id_user:id_user+batch_size]
    v0 = train_set[id_user:id_user+batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0,vk,ph0,phk)
    train_loss += torch.mean(torch.abs(vk[vk>=0]-v0[v0>=0]))
    s += 1
  print(f'epochs->{epoch}\t loss->{train_loss/s}')

epochs->1	 loss->0.08400315791368484
epochs->2	 loss->0.016311930492520332
epochs->3	 loss->0.018224336206912994
epochs->4	 loss->0.01960364729166031
epochs->5	 loss->0.02043665014207363
epochs->6	 loss->0.020148631185293198
epochs->7	 loss->0.022194476798176765
epochs->8	 loss->0.0210272166877985
epochs->9	 loss->0.02179349958896637
epochs->10	 loss->0.02186550199985504


In [132]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.8411)


In [134]:
import numpy as np
u = np.random.choice([0,1], 100000)
v = np.random.choice([0,1], 100000)
u[:50000] = v[:50000]
sum(u==v)/float(len(u)) # -> you get 0.75
np.mean(np.abs(u-v)) # -> you get 0.25

0.25115

In [135]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2)) # RMSE here
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.7411)
epoch: 2 loss: tensor(0.5678)
epoch: 3 loss: tensor(0.5294)
epoch: 4 loss: tensor(0.5166)
epoch: 5 loss: tensor(0.5171)
epoch: 6 loss: tensor(0.5160)
epoch: 7 loss: tensor(0.5122)
epoch: 8 loss: tensor(0.5061)
epoch: 9 loss: tensor(0.5120)
epoch: 10 loss: tensor(0.5137)


In [136]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2)) # RMSE here
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.5059)
